# "Modulo 5"
> "Analisi DMRG del modello di Ising 1D"

A differenza dei precedenti moduli in questa relazione non mi occuperò di studiare sistemi con interazioni a lunga distanza random poiché il DMRG si basa fondamentalmente su interazioni a primi vicini. Sono possibili estensioni per considerare interazioni a lungo raggio, ma richiedono un notevole sforzo sia di implementazione che computazionale.

La **Density Matrix Renormalization Group** è una tecnica numerica iterativa che consente di trovare il ground state, ed eventualmente pochi altri stati eccitati, di un sistema quantistico a bassa dimensionalità in una maniera estremamente efficiente. 
È un metodo approssimato che si ispira alla rinormalizzazione numerica alla Wilson, ma il cui funzionamento si basa sull'*entanglement bipartito* per il ground state di $\hat{H}$, condizione sufficiente per la validità dell'*Area Law*. Considerando uno stato pure $|\psi⟩_{AB}$ di un sistema quantistico bipartito AB, questa proprietà, dimostrata per il caso unidimensionale e come congettura per il caso 2D, esprime la dipendenza lineare tra l'entropia di Von Neumann della partizione A e la dimensione del confine tra A e B.
$$ S(\rho_A) \sim dim(bound(A|B))$$
in cui 
$$\rho_A = Tr_B(|\psi⟩_{AB}⟨\psi|)$$






Una generica Hamiltoniana che soddisfa queste condizioni può essere scritta come:

### $$ \hat{H}=\sum_{i=1}^L ( \sum_\alpha J_i^{(\alpha)} \hat{S}_i^{(\alpha)} \hat{T}_{i+1}^{(\alpha)} +\sum_\beta B_i^{(\beta)} \hat{V}_i^{(\beta)} ) $$

L'Hamiltoniana del modello di Ising quantistico 1D rientra in questa forma:

## $$ \hat{H} = -\sum_{i}^L \sigma_i^z \sigma_{i+1}^z + g \sum_{i}^L \sigma_i^x $$

L'algoritmo del DMRG si articola in **alcuni** step:
1. Si parte da un blocco $B(1,d)$, composto dal solo sito estremo di sinistra, di cui si definisce l'Hamiltoniana $\hat{H}_B$, nel codice *BlockH*. Spazio di Hilbert di dimensione $d$.
2. Si costruisce l'*Enlarged Block* aggiungendo al blocco precedente il sito adiacente destro e si costruisce l'hamiltoniana corrispondente $\hat{H}_E$:
$$ \hat{H}_E = \hat{H}_B \otimes \mathbb{1}_{sito} + \mathbb{1}_B \otimes \hat{H}_{sito} + \hat{H}_{B-sito} $$ Spazio di Hilbert di dimensione $d^2$.
3. Si costruisce il *Super-block* aggiundendo al blocco precedente un blocco speculare, considerando il fatto che il sistema in esame è simmetrico per riflessione rispetto al centro della catena, il collegamento è dato dall'interazione dei due siti esterni aggiunti al passo precedente. L'Hamiltoniana del *Super-block* diventa:
$$ \hat{H}_{SB} = \hat{H}_E \otimes \mathbb{1}_{E'} + \mathbb{1}_E \otimes \hat{H}_{E'} + \hat{H}_{E-E'} $$

In [3]:
#collapse
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse.linalg as ssl
import scipy.sparse as ss

In [2]:
#%%time
m=10
NIter=100
rep=10
rep+=1
gmax=2
ling=np.linspace(0,gmax,rep)


nrip=10

#ling=ling[int(30*rep/100):]
Evec=np.zeros(rep)
Entropy=np.zeros(rep)
ggg=0


# inizialize local ops

I=np.eye(2,2)
Sz=np.array([[1, 0],[0,-1]])
Sx=np.array([[0, 1],[1,0]])

# initial blocks
for gm in ling:
    #ggg=0
    Ent=0
    for rip in range(nrip):
        BlockSz = Sz
        BlockSx = Sx
        BlockI  = I
        #BlockH  = gm*Sx
        BlockH = np.random.default_rng().normal(gm, .8)*Sx
        Energy = 0

        for l in range(NIter):
            SystSize = 2*l + 4

            g=np.random.default_rng().normal(gm, .8)
            #g=gm
            
        # Get the 2m-dimensional operators for the block + site

            BlockH = np.kron(BlockH, I) + np.kron(BlockI, g*Sx) - np.kron(BlockSz, Sz) 
            BlockSz = np.kron(BlockI, Sz)
            BlockSx = np.kron(BlockI, Sx)
            BlockI  = np.kron(BlockI, I)

        # HAMILTONIAN MATRIX for superblock
            H_super = np.kron(BlockH, BlockI) + np.kron(BlockI, BlockH) - np.kron(BlockSz, BlockSz)
            H_super = 0.5 * (H_super + H_super.T);  # ensure H is symmetric

        # Diagonalizing the Hamiltonian
            LastEnergy = Energy
            Energy= ssl.eigsh(H_super,1,which='SA')[0] #[0]
            Psi = ssl.eigsh(H_super,1,which='SA')[1].T #[0]
            EnergyPerBond = (Energy - LastEnergy) / 2
            Ener2 = Energy / SystSize


        # Sigma = Psi' *kron(BlockSz,BlockSz) * Psi; % n.n. ZZ correlation function

        # Form the reduced density matrix
            nr=Psi.size
            Dim = int(np.sqrt(nr))
            PsiMatrix = np.reshape(Psi,(Dim,Dim))
            Rho = PsiMatrix @ PsiMatrix.T

        # Diagonalize the density matrix
            D,V = np.linalg.eigh(Rho)
            D=D[::-1]  # descending
            Index=np.arange(Dim)
            Index=Index[::-1]
            V=V[:,Index]

        # Construct the truncation operator
            NKeep = min(D.size, m)

            Omatr = V[:,:NKeep]

            TruncationError = 1 - sum(D[:NKeep])


                # Transform the block operators into the truncated basis
            BlockH  = Omatr.T @ BlockH  @ Omatr
            BlockSz = Omatr.T @ BlockSz @ Omatr
            BlockSx = Omatr.T @ BlockSx @ Omatr
            BlockI  = Omatr.T @ BlockI  @ Omatr


            
        #print(SystSize, Energy, EnergyPerBond, Ener2, TruncationError)

        Ent+=-sum(D[:NKeep]*np.log(D[:NKeep]))
        #Evec[ggg]=Energy
    Entropy[ggg]=Ent/nrip
    ggg+=1
    
    
    #Spectrum=ssl.eigsh(H_super)[0]
    
plt.plot(ling,Entropy)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
np.random.default_rng().normal(gm, .2)

-0.19960819322372814

In [41]:
Psi.shape

(1, 200)

In [ ]:
EMIndex=np.argmax(Entropy)+1
ling=np.linspace(0,gmax,rep)
ling=(ling-EMIndex/rep*gmax)**3+(EMIndex/rep*gmax)**3
Evec=np.zeros(rep)
Entropy=np.zeros(rep)
ggg=0
    
for g in ling:
    BlockSz = Sz
    BlockSx = Sx
    BlockI  = I
    BlockH  = g*Sx
    Energy = 0

    #for g in ling:

    for l in range(NIter):
        SystSize = 2*l + 4

    # Get the 2m-dimensional operators for the block + site

        BlockH = np.kron(BlockH, I) + np.kron(BlockI, g*Sx) - np.kron(BlockSz, Sz) 
        BlockSz = np.kron(BlockI, Sz)
        BlockSx = np.kron(BlockI, Sx)
        BlockI  = np.kron(BlockI, I)

    # HAMILTONIAN MATRIX for superblock
        H_super = np.kron(BlockH, BlockI) + np.kron(BlockI, BlockH) - np.kron(BlockSz, BlockSz)
        H_super = 0.5 * (H_super + H_super.T);  # ensure H is symmetric

    # Diagonalizing the Hamiltonian
        LastEnergy = Energy
        Energy= ssl.eigsh(H_super,1,which='SA')[0] #[0]
        Psi = ssl.eigsh(H_super,1,which='SA')[1].T #[0]
        EnergyPerBond = (Energy - LastEnergy) / 2
        Ener2 = Energy / SystSize


    # Sigma = Psi' *kron(BlockSz,BlockSz) * Psi; % n.n. ZZ correlation function

    # Form the reduced density matrix
        nr=Psi.size
        Dim = int(np.sqrt(nr))
        PsiMatrix = np.reshape(Psi,(Dim,Dim))
        Rho = PsiMatrix @ PsiMatrix.T

    # Diagonalize the density matrix
        D,V = np.linalg.eigh(Rho)
        D=D[::-1]  # descending
        Index=np.arange(Dim)
        Index=Index[::-1]
        V=V[:,Index]

    # Construct the truncation operator
        NKeep = min(D.size, m)
        Ent=-sum(D[:NKeep]*np.log(D[:NKeep]))
        Omatr = V[:,:NKeep]
        TruncationError = 1 - sum(D[:NKeep])

    # Transform the block operators into the truncated basis
        BlockH  = Omatr.T @ BlockH  @ Omatr
        BlockSz = Omatr.T @ BlockSz @ Omatr
        BlockSx = Omatr.T @ BlockSx @ Omatr
        BlockI =  Omatr.T @ BlockI  @ Omatr

        #print(SystSize, Energy, EnergyPerBond, Ener2, TruncationError)
    
    Evec[ggg]=Energy
    Entropy[ggg]=Ent
    ggg+=1

In [ ]:
    LBlockH = BlockH
    RBlockH = BlockH
    
    for l in range(NIter2):
        
        LBlockH = np.kron(BlockH, I) + np.kron(BlockI, g*Sx) - np.kron(BlockSz, Sz) 
        BlockSz = np.kron(BlockI, Sz)
        BlockSx = np.kron(BlockI, Sx)
        BlockI  = np.kron(BlockI, I)
        
        # HAMILTONIAN MATRIX for superblock
        LH_super = np.kron(LBlockH, BlockI) + np.kron(BlockI, BlockH) - np.kron(BlockSz, BlockSz)
        H_super = 0.5 * (H_super + H_super.T);  # ensure H is symmetric
        

In [ ]:
# Form the reduced density matrix
        nr=Psi.size
        Dim = int(np.sqrt(nr))
        PsiMatrix = np.reshape(Psi,(Dim,Dim))
        Rho = PsiMatrix @ PsiMatrix.T

    # Diagonalize the density matrix
        D,V = np.linalg.eigh(Rho)
        D=D[::-1]  # descending
        Index=np.arange(Dim)
        Index=Index[::-1]
        V=V[:,Index]

    # Construct the truncation operator
        NKeep = min(D.size, m)
        
        Omatr = V[:,:NKeep]
        
        TruncationError = 1 - sum(D[:NKeep])

    # Transform the block operators into the truncated basis
        BlockH  = Omatr.T @ LBlockH  @ Omatr
        BlockSz = Omatr.T @ LBlockSz @ Omatr
        BlockSx = Omatr.T @ LBlockSx @ Omatr
        BlockI  = Omatr.T @ LBlockI  @ Omatr

